# Npay증권에서 주가정보 수집하기

In [1]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
import os
from datetime import datetime
from sqlalchemy import create_engine, text
import pymysql
pymysql. install_as_MySQLdb()
load_dotenv(dotenv_path="./data/.env_db")

True

In [2]:
db_id = os.getenv("id")
db_pw = os.getenv("pw")
addr = os.getenv("addr")
port = os.getenv("port")
engine = create_engine(f"mysql+pymysql://{db_id}:{db_pw}@{addr}:{port}/stock_info_20251022")
conn = engine.connect()

In [3]:
data = pd.read_csv("./data/상장기업정보2.csv")
data

,주식종목,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,명인제약,317450,의약품 제조업,"신경정신계열, 구강계열 완제의약품",2025-10-01,12월,이행명,http://www.myunginph.co.kr,경기도
1,코스닥,KB제33호스팩,0072Z0,금융 지원 서비스업,기업인수합병,2025-09-30,12월,박성원,홈페이지 없음,서울특별시
2,코스닥,삼성스팩11호,0071M0,금융 지원 서비스업,기업인수합병,2025-09-30,12월,문소연,홈페이지 없음,서울특별시
3,코스닥,에스투더블유,488280,소프트웨어 개발 및 공급업,"공공/정부기관용 사이버안보 빅데이터 플랫폼, 기업 및 기관용 사이버위협 인텔리전스 ...",2025-09-19,12월,서상덕,http://s2w.inc/,경기도
4,코스닥,오아,342870,가전제품 및 정보통신장비 소매업,계절/생활/주방/건강가전 및 헬스/건강식품,2025-09-09,12월,김상무,http://www.oa-world.com,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2758,유가증권,유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2759,유가증권,CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2760,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2761,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


# dbio를 이용해서 DB에서 데이터 불러오기

In [2]:
import re
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
from dotenv import load_dotenv
import os
from datetime import datetime
from dbio import load_data, to_db

In [3]:
data = load_data("stock_info_20251022", "stock_company_list20251022")

In [9]:
data

,주식종목,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,유가증권,명인제약,317450,의약품 제조업,"신경정신계열, 구강계열 완제의약품",2025-10-01,12월,이행명,http://www.myunginph.co.kr,경기도
1,코스닥,KB제33호스팩,0072Z0,금융 지원 서비스업,기업인수합병,2025-09-30,12월,박성원,홈페이지 없음,서울특별시
2,코스닥,삼성스팩11호,0071M0,금융 지원 서비스업,기업인수합병,2025-09-30,12월,문소연,홈페이지 없음,서울특별시
3,코스닥,에스투더블유,488280,소프트웨어 개발 및 공급업,"공공/정부기관용 사이버안보 빅데이터 플랫폼, 기업 및 기관용 사이버위협 인텔리전스 ...",2025-09-19,12월,서상덕,http://s2w.inc/,경기도
4,코스닥,오아,342870,가전제품 및 정보통신장비 소매업,계절/생활/주방/건강가전 및 헬스/건강식품,2025-09-09,12월,김상무,http://www.oa-world.com,서울특별시
...,...,...,...,...,...,...,...,...,...,...
2758,유가증권,유한양행,000100,의약품 제조업,"의약품(삐콤씨, 안티푸라민, 렉라자, 로수바미브, 코푸시럽 등), 생활용품(유한락스...",1962-11-01,12월,대표이..,http://www.yuhan.co.kr,서울특별시
2759,유가증권,CJ대한통운,000120,도로 화물 운송업,"Contract Logistics, 포워딩, 항만하역, 해운, 택배국제특송, SCM...",1956-07-02,12월,신영수..,http://www.cjlogistics.com,서울특별시
2760,유가증권,경방,000050,종합 소매업,"섬유류(면사,면혼방사,면직물,면혼방직물,화섬사,화섬직물) 제조,도매,수출입",1956-03-03,12월,"김준,..",http://www.kyungbang.co.kr,서울특별시
2761,유가증권,유수홀딩스,000700,회사 본부 및 경영 컨설팅 서비스업,지주사업,1956-03-03,12월,송영규,http://www.eusu-holdings.com,서울특별시


1. 종목명, 현재가, 변동금액, 변동률, 전일가, 고가, 상한가, 저가, 하한가, 거래량 찾기
2. 데이터프레임으로 만들기 + 오늘 날짜 컬럼 추가
3. mysql에 저장하기table이름은 stock_price_info

# 주가 정보 수집코드를 함수화, 1개 데이터를 바로 DB저장

In [7]:
today = datetime. today()
today = f"{today.year }{today.month}{today.day}"
today

'20251022'

In [8]:
def naver_stock(code):
    result = {}
    cols = ["수집일","종목명","종목코드","현재가","변동금액","변동률","전일가","시가","고가","상한가","저가","하한가","거래량"]

    today = datetime. today()
    today = f"{today.year }{today.month}{today.day}"
    
    url = f"https://finance.naver.com/item/main.naver?code={code}"
    r = requests.get(url)
    soup = bs(r.content, 'lxml')

    # 종목명
    stock_name = soup.select_one("dl.blind > dd:nth-child(3)").text.split()[1]
    # 현재가
    today_price = soup.select_one("dl.blind > dd:nth-child(5)").text.split()[1]
    # 변동금액
    change = soup.select_one("dl.blind > dd:nth-child(5)").text.split()[3:5]
    change = -int(change[1].replace(",","")) if change[0] == '하락'else int(change[1].replace(",",""))
    # 상승인지 하락인지 확인하고 +,- 붙여줘야함!
    # 변동률
    percent = soup.select_one("dl.blind > dd:nth-child(5)").text.split()[5:8]
    percent = "".join(percent)
    percent = percent.replace("플러스", "").replace("마이너스", "-").replace("퍼센트", "%")
    # 전일가
    yester_price = int(soup.select_one("dl.blind > dd:nth-child(6)").text.split()[1].replace(",",""))
    # 시가
    opening_price = int(soup.select_one("dl.blind > dd:nth-child(7)").text.split()[1].replace(",",""))
    # 고가
    high = int(soup.select_one("dl.blind > dd:nth-child(8)").text.split()[1].replace(",",""))
    # 상한가
    top = int(soup.select_one("dl.blind > dd:nth-child(9)").text.split()[1].replace(",",""))
    # 저가
    low = int(soup.select_one("dl.blind > dd:nth-child(10)").text.split()[1].replace(",",""))
    # 하한가
    bottom = int(soup.select_one("dl.blind > dd:nth-child(11)").text.split()[1].replace(",",""))
    # 거래량
    volume = int(soup.select_one("dl.blind > dd:nth-child(12)").text.split()[1].replace(",",""))

    values = (today, stock_name, code, today_price, change, percent, yester_price, opening_price,
     high, top, low, bottom, volume)

    for key,value in zip(cols, values):
        result.setdefault(key,[]).append(value)
        
    df = pd.DataFrame(result)

    to_db("stock_info_20251022", f"stock_price_{today.year}", df)
    return print(f"{stock_name} 주가정보 수집완료         ", end="Wr")


# 데이터 수집 및 저장하기

In [9]:
for idx, code in enumerate (data['종목코드']):
    print(f"{idx+1}/{len(data['종목코드'])} 데이터 수집중", end="\r")
    try:
        int(code[:3])
        naver_stock(code)
    except Exception as e:
        print(e)

'str' object has no attribute 'year'
'str' object has no attribute 'year'
'str' object has no attribute 'year'
'str' object has no attribute 'year'
'str' object has no attribute 'year'


KeyboardInterrupt: 